# Process the user location 1.3 billion record file

Activity will remain in spark - especially RDDs.

The process will follow this outlined list:

  1. read in user 1.3 billion record file - keep only 4 cols - userid, event_date, latitude and longitude
  
  1. build a kd-tree on a subest - this is done pre prod on 1.2 million records.
  
  1. use the kd-tree leaves to create the partitions of the user RDD
  
  1. read in the poi table - into an RDD
  
  1. distribute the poi table to all partitions
  
  1. loop through poi table, using the poi points to select all user points within a radius of category_distance
  
  1. retain and write out to csv the resulting user location data.

In [1]:
from shapely.geometry import *
import fiona 
import geospark
import pandas as pd
import geopandas
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import numpy as np
from matplotlib.cm import *
from IPython.display import Image as ImageShow
from pyspark.sql import *
import pyspark.sql.functions as f
import pyspark.sql.types as types
from pyspark.sql.types import *
from math import radians, cos, sin, asin, sqrt
from geospark.sql.types import GeometryType
from geopandas import GeoDataFrame 
import rtree
from scipy.spatial import cKDTree
from geospark.core.SpatialRDD import PointRDD
from geospark.core.enums import FileDataSplitter

In [2]:
from geospark.register import GeoSparkRegistrator
from geospark.register import upload_jars
from geospark.core.SpatialRDD import SpatialRDD
from geospark.core.SpatialRDD import PointRDD
from geospark.core.SpatialRDD import PolygonRDD
from geospark.core.SpatialRDD import LineStringRDD
from geospark.core.enums import FileDataSplitter
from geospark.utils.adapter import Adapter
from geospark.core.spatialOperator import KNNQuery
from geospark.core.spatialOperator import JoinQuery
from geospark.core.spatialOperator import RangeQuery
from geospark.core.formatMapper.shapefileParser import ShapefileReader
from geospark.core.formatMapper import WkbReader
from geospark.core.formatMapper import WktReader
from geospark.core.formatMapper import GeoJsonReader
from geospark.sql.types import GeometryType
from geospark.core.enums import GridType
from geospark.core.SpatialRDD import RectangleRDD
from geospark.core.enums import IndexType
from geospark.core.geom.envelope import Envelope
from geospark.utils import GeoSparkKryoRegistrator, KryoSerializer

In [13]:

spark = SparkSession.\
    builder.\
    master("local[6]").\
    appName("Crossover").\
    getOrCreate()

In [3]:

spark = SparkSession.\
    builder.\
    master("local[6]").\
    appName("Crossover").\
    config("spark.serializer", KryoSerializer.getName).\
    config("spark.kryo.registrator", GeoSparkKryoRegistrator.getName) .\
    getOrCreate()


In [4]:
spark

In [5]:
GeoSparkRegistrator.registerAll(spark)

Py4JJavaError: An error occurred while calling o47.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: org.apache.spark.SparkException: Failed to register classes with Kryo
org.apache.spark.SparkException: Failed to register classes with Kryo
	at org.apache.spark.serializer.KryoSerializer.newKryo(KryoSerializer.scala:140)
	at org.apache.spark.serializer.KryoSerializerInstance.borrowKryo(KryoSerializer.scala:328)
	at org.apache.spark.serializer.KryoSerializerInstance.<init>(KryoSerializer.scala:313)
	at org.apache.spark.serializer.KryoSerializer.newInstance(KryoSerializer.scala:222)
	at org.apache.spark.broadcast.TorrentBroadcast$.blockifyObject(TorrentBroadcast.scala:288)
	at org.apache.spark.broadcast.TorrentBroadcast.writeBlocks(TorrentBroadcast.scala:127)
	at org.apache.spark.broadcast.TorrentBroadcast.<init>(TorrentBroadcast.scala:88)
	at org.apache.spark.broadcast.TorrentBroadcastFactory.newBroadcast(TorrentBroadcastFactory.scala:

In [16]:
conf.set("spark.serializer", KryoSerializer.getName)
conf.set("spark.kryo.registrator", GeoSparkKryoRegistrator.getName)

NameError: name 'conf' is not defined

## Read in user_location file

In AWS EMR cluster this will be from S3.  

In [4]:
user_schema = types.StructType([StructField('userid', StringType(), False),
                        StructField('event_date', StringType(), True),
                        StructField('latitude', DecimalType(18,15), False),
                        StructField('longitude', DecimalType(18,15), False),
                        StructField('round_longitude', DecimalType(6,2), False),  
                        StructField('round_latitude', DecimalType(6,2), True),
                        StructField('sround_longitude', DecimalType(9,5), True),
                        StructField('sround_latitude', DecimalType(9,5), True),
                        StructField('bround_longitude', DecimalType(9,5), True),
                        StructField('bround_latitude', DecimalType(9,5), True),
                        StructField('mround_longitude', DecimalType(9,5), True),
                        StructField('mround_latitude', DecimalType(9,5), True),
                        StructField('geopoint', StringType(), True),
                        StructField('postcode', StringType(), True)])

user_raw = spark.read.format('csv')  \
                .schema(user_schema)  \
                .load("./xm_ls_26thMay_for_beatrice*.csv")

user_spark = user_raw.select([c for c in user_raw.columns if c in ['userid','event_date','latitude','longitude']])

user_spark.show()

+--------------------+--------------------+------------------+------------------+
|              userid|          event_date|          latitude|         longitude|
+--------------------+--------------------+------------------+------------------+
|000019e5-4a53-48a...|2019-05-23 09:53:...|53.397247314453125|-2.497413192183255|
|00002c68-089b-4e9...|2019-05-23 12:31:...|51.919342041015625|-0.699813561514671|
|00002c68-089b-4e9...|2019-05-23 12:31:...|51.919342041015625|-0.699813561514671|
|00007903-4cae-4e0...|2019-05-23 10:17:...|52.618834700000000|-1.073803800000000|
|00007903-4cae-4e0...|2019-05-23 10:17:...|52.618834700000000|-1.073803800000000|
|00007903-4cae-4e0...|2019-05-23 10:17:...|52.618834700000000|-1.073803800000000|
|00007903-4cae-4e0...|2019-05-23 11:59:...|52.637397700000000|-1.072081700000000|
|00007903-4cae-4e0...|2019-05-23 11:59:...|52.637397700000000|-1.072081700000000|
|00007903-4cae-4e0...|2019-05-23 11:59:...|52.637397700000000|-1.072081700000000|
|00007903-4cae-4

In [5]:
print(user_spark.rdd.getNumPartitions())
#print(df.rdd.getNumPartitions())

#user_spark.groupBy(f.spark_partition_id).count

user_spark.withColumn("partitionId", f.spark_partition_id()).groupBy("partitionId").count().show()

6
+-----------+------+
|partitionId| count|
+-----------+------+
|          1|353418|
|          3|339747|
|          5|324568|
|          4|333593|
|          2|353212|
|          0|353223|
+-----------+------+



In [6]:
x = user_spark.select("longitude", "latitude").collect()


In [7]:
len(x)

2057761

## Read in poi file

In [8]:
poi_schema = types.StructType([StructField('poi_id', IntegerType(), False),
                         StructField('brand', StringType(), True),
                         StructField('category', StringType(), False),
                         StructField('category_group', StringType(), False), 
                         StructField('poi_lat', DecimalType(18,15), True),
                         StructField('poi_long', DecimalType(18,15), True),
                         StructField('category_distance', IntegerType(), True),
                         StructField('hidden', StringType(), True),
                         StructField('region', StringType(), True)])

poi_raw = spark.read.format('csv')  \
                .schema(poi_schema)  \
                .load("./poi_cleansed.csv",header='true')

poi_raw.count()

255977

In [9]:
user_kdtree = cKDTree(x,copy_data=True)

In [10]:
user_kdtree.tree.data_points

array([[-6.338326  , 49.89437976],
       [-6.3354152 , 49.89435766],
       [-6.33944413, 49.89438066],
       ...,
       [-1.2592082 , 60.4480301 ],
       [-1.4070859 , 60.46765137],
       [-1.4070859 , 60.46765137]])

In [18]:
#number of points in each leaf
user_kdtree.data.getfield(1)


TypeError: data type not understood

In [12]:
#number of nodes in the tree.
user_kdtree.size


528691

In [13]:
lx = user_kdtree.tree.indices.min
print(lx)

<built-in method min of numpy.ndarray object at 0x7f8074e33a80>


In [69]:
user_kdtree.tree.lesser.children

1028877

In [67]:
user_kdtree.tree.greater.children

1028884

In [73]:
user_kdtree.tree.lesser.lesser.children

514437

In [74]:
user_kdtree.tree.lesser.greater.children

514440

In [72]:
r = user_kdtree.mins
r

array([-8.1190568 , 49.88896216])

In [73]:
user_kdtree.maxes

array([ 1.76004718, 60.46765137])

In [11]:
partition_parts = user_kdtree.tree.data_points
print(partition_parts)

[[-6.338326   49.89437976]
 [-6.3354152  49.89435766]
 [-6.33944413 49.89438066]
 ...
 [-1.2592082  60.4480301 ]
 [-1.4070859  60.46765137]
 [-1.4070859  60.46765137]]


Special partition by the partition_parts array

In [ ]:
user_partition = user_spark.repartitionByRange()

In [ ]:
test_series = pd.Series( kdtree_counts[1].ravel() ).value_counts()

In [26]:
user_partition = user_spark.repartition(10000,"longitude","latitude")

In [27]:

user_partition.withColumn("partitionId", f.spark_partition_id()).groupBy("partitionId").count().show()


+-----------+-----+
|partitionId|count|
+-----------+-----+
|        148|  201|
|        463|  205|
|        471|  205|
|        496|  187|
|        833|  197|
|       1088|  176|
|       1238|  203|
|       1342|  168|
|       1580|  269|
|       1591|  214|
|       1645|  186|
|       1829|  186|
|       1959|  170|
|       2122|  185|
|       2142|  198|
|       2366|  198|
|       2659|  197|
|       2866|  210|
|       3175|  168|
|       3749|  196|
+-----------+-----+
only showing top 20 rows



In [30]:
print("Partitions structure: {}".format(user_partition.rdd.glom().collect()))

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 43040)
Traceback (most recent call last):
  File "/usr/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/local/lib/python3.7/dist-packages/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/lib/python3.7/dist-packages/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/lib/python3.7/dist-packages/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/lo

In [29]:
#-0.06867999999999999,51.38215

no_of_points = len(user_kdtree.query_ball_point([-0.06867999999999999,51.38215], 0.002))
no_of_points

#radius = 2 1 351 896
#radius = 0.2 432 814
#radius = 0.002 48 points

48

In [67]:
xx = user_kdtree.query_ball_point([-0.06867999999999999,51.38215], 0.002)
print(xx)

[40353, 210479, 1064793, 210480, 1064794, 1049089, 1006310, 1006301, 1006305, 1006314, 1006308, 1006304, 1006309, 1006315, 1006316, 1006307, 1006306, 1006302, 1006311, 1006312, 1006313, 1006303, 1607415, 1666128, 1954274, 720382, 720381, 720380, 652919, 2021433, 652918, 1957075, 1594131, 1594132, 1594130, 1594133, 85128, 1381687, 1025749, 786997, 786996, 454268, 1745740, 929568, 1867308, 585101, 585100, 585099]


In [54]:
user_kdtree[1].ravel()

TypeError: 'scipy.spatial.ckdtree.cKDTree' object is not subscriptable

In [ ]:
51.599243164062500,-0.196039105416565